# Exercise 6

## SETUP

In [1]:
import os
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import MSELoss, L1Loss
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchvision
from torchvision.io import read_image
import torchvision.transforms as transforms
from torchvision.io import write_png

from pytorch_msssim import SSIM

c:\Users\omera\.conda\envs\ML2\lib\site-packages\torchvision\models\detection\anchor_utils.py:63: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  device: torch.device = torch.device("cpu"),


In [2]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU
else:
    device = torch.device("cpu")  # Use CPU

# Print the device being used
print("Device:", device)

Device: cuda


In [3]:
# Define class SRDataset
class SRDataset(Dataset):
    def __init__(self, folder_path, augment):
        self.folder_path = folder_path
        self.image_filenames = os.listdir(folder_path)
        self.t_crop = transforms.Compose([transforms.RandomCrop(64)])
        self.t_colorjitter = transforms.Compose([transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)])
        self.t_downscale = transforms.Compose([transforms.Resize((32, 32), interpolation=transforms.InterpolationMode.BILINEAR, antialias=True)])
        self.augment = augment
        
    def __len__(self):
        return len(self.image_filenames)
    
    def __getitem__(self, index):
        image_path = os.path.join(self.folder_path, self.image_filenames[index])
        hr_image = self.t_crop(read_image(image_path) / 255.0)  # Convert to float between 0 and 1
        if(self.augment):
            lr_image = self.t_downscale(self.t_colorjitter(hr_image))
        else:
            lr_image = self.t_downscale(hr_image)
        
        return lr_image, hr_image


In [4]:
# # Define Model
# class BasicSRModel(nn.Module):
#     def __init__(self, num_inter_blocks):
#         super(BasicSRModel, self).__init__()
        
#         self.conv_blocks = nn.Sequential(nn.ConvTranspose2d(3, 64, kernel_size=4, stride=2, padding=1))
            
#         for i in range(num_inter_blocks):  # Number of intermediate blocks
#             self.conv_blocks.add_module(
#                 f"conv_{i+1}",
#                 nn.Sequential(
#                     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
#                     nn.LeakyReLU(inplace=True),
#                 )
#             )
        
#         self.conv_blocks.add_module(
#             "last_conv",
#             nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)
#         )
    
#     def forward(self, x):
#         x = self.conv_blocks(x)
#         return x

In [5]:
# Define Model
class BasicSRModel(nn.Module):
    def __init__(self, num_inter_blocks):
        super(BasicSRModel, self).__init__()

        
        self.conv_blocks = nn.Sequential(nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False))
        self.conv_blocks.add_module(
            "first_conv",
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        )


        #self.conv_blocks = nn.Sequential(nn.ConvTranspose2d(3, 64, kernel_size=4, stride=2, padding=1))
            
        for i in range(num_inter_blocks):  # Number of intermediate blocks
            self.conv_blocks.add_module(
                f"conv_{i+1}",
                nn.Sequential(
                    nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
                    nn.LeakyReLU(inplace=True),
                )
            )
        
        self.conv_blocks.add_module(
            "last_conv",
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)
        )
    
    def forward(self, x):
        upscaled_image = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)(x)
        output = self.conv_blocks(x)
        output += upscaled_image  # Add the upscaled image to the output as residual
        
        return output
    
# Create an instance of BasicSRModel
model = BasicSRModel(10)

# Count the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(num_params)


372803


## TRAINING & EVALUATION

In [6]:
# Load and initialize the train_dataset
train_datapath = os.path.join(os.path.abspath(''), 'data/train')
train_dataset = SRDataset(train_datapath, augment=True)
train_batch_size = 4
train_dataloader = DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    )

In [7]:
# Load and initialize the test_dataset
test_datapath = os.path.join(os.path.abspath(''), 'data/eval')
test_dataset = SRDataset(test_datapath, augment=False)
test_batch_size = 9
test_dataloader = DataLoader(
    test_dataset,
    batch_size=test_batch_size,
    shuffle=True,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    )

In [8]:
# Create an instance of BasicSRModel
model.to(device)

# Define optimizer
learning_rate = 1e-4
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr=learning_rate)

# Define loss function
loss_function = L1Loss()
loss_function.to(device)

# # Print the model architecture
# print(model)

# # Check number of parameters in model
num_params = 0
for param in model.parameters():
    num_params += param.numel()
print("num_params: " + str(num_params))


num_params: 372803


In [9]:
writer = SummaryWriter()
number_of_epochs = 500
for epoch in range(number_of_epochs):
    with tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{number_of_epochs}', unit='batch') as tqdm_train_dataloader:
        # TRAIN BATCH
        cum_loss = 0
        for _, (lr_image, hr_image) in enumerate(tqdm_train_dataloader):
            lr_image, hr_image = lr_image.to(device), hr_image.to(device)
            # reset the gradient
            optimizer.zero_grad()
            # forward pass through the model
            hr_prediction = model(lr_image)  
            # compute the loss
            loss = loss_function(hr_prediction, hr_image)
            # backpropagation
            loss.backward()
            # update the model parameters
            optimizer.step()
            # add loss to be displayed at the end of epoch
            cum_loss += loss.item()
        # log training loss
        writer.add_scalar('loss/train', cum_loss / train_batch_size, epoch)


        # EVALUATE BATCH
        cum_l1 = 0.0
        cum_psnr = 0.0
        cum_ssim = 0.0
        with torch.no_grad():
            for _, (lr_image, hr_image) in enumerate(test_dataloader):
                lr_image, hr_image = lr_image.to(device), hr_image.to(device)
                hr_prediction = model(lr_image)
                # L1
                l1_metric = L1Loss()
                l1_metric.to(device)
                l1_i = l1_metric(hr_prediction, hr_image)
                # PSNR
                mse_metric = MSELoss()
                mse_metric.to(device)
                psnr_i = -10 * torch.log10(mse_metric(hr_prediction, hr_image))
                # SSIM
                ssim_metric = SSIM(data_range=1.0)
                ssim_metric.to(device)
                ssim_i = ssim_metric(hr_prediction, hr_image)
                # accumulate metrics
                cum_psnr += psnr_i.item()
                cum_ssim += ssim_i.item() 
                cum_l1 +=  l1_i.item()
            # Log test loss
            writer.add_scalar('loss/test-l1', cum_l1 / test_batch_size, epoch)   
            writer.add_scalar('loss/test-psnr', cum_psnr / test_batch_size, epoch)
            writer.add_scalar('loss/test-ssim', cum_ssim / test_batch_size, epoch)   
    

Epoch 500/500: 100%|██████████| 75/75 [00:02<00:00, 30.37batch/s]


In [10]:
# Save model
torch.save(model.state_dict(), "saved-models/model-residual-500.pt")

In [11]:
# #Load model
# model = BasicSRModel(10)
# model.load_state_dict(torch.load("saved-models/modelv2-500.pt"))
# model.to(device); # Suppress output

In [50]:
# Save images for comparison
with torch.no_grad():
    for _, (lr_image, hr_image) in enumerate(test_dataloader):
        lr_image, hr_image = lr_image.to(device), hr_image.to(device)
        hr_prediction = model(lr_image)
        
        # Display images
        lr_image_disp = lr_image.to('cpu')
        hr_image_disp = hr_image.to('cpu')
        hr_prediction_disp = hr_prediction.to('cpu')
        write_png(lr_image_disp[0, ...].mul(255).byte(), "image-outputs/lr_image.png")
        write_png(hr_image_disp[0, ...].mul(255).byte(), "image-outputs/hr_image.png")
        write_png(hr_prediction_disp[0, ...].mul(255).byte(), "image-outputs/hr_prediction.png")

        # Upscale using analytical upscaling methods
        hr_bilinear = transforms.functional.resize(lr_image_disp.squeeze(0), (64, 64), interpolation=2)
        hr_bicubic = transforms.functional.resize(lr_image_disp.squeeze(0), (64, 64), interpolation=3)
        write_png(hr_bilinear[0, ...].mul(255).byte(), "image-outputs/bilinear.png")
        write_png(hr_bicubic[0, ...].mul(255).byte(), "image-outputs/bicubic.png") 
        break

![alternative text](D:\VSCode Git Repos\MFCGV-Projects\MF2023-Exercise6\selected-images\graph_epochs500.png)